In [ ]:
# Documentação
# https://python-twitter.readthedocs.io/en/latest/twitter.html#module-twitter.api

## Importando bibliotecas

In [1]:
!pip install python-twitter

In [2]:
import twitter
import json

In [3]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

## Configurando api

In [ ]:
# Variaveis de acesso, keys cedidas pela api do twitter
# tokens.json

In [ ]:
# Carregando o arquivo de configuração
with open('tokens.json') as f:
    data = json.load(f)

In [ ]:
# Carregando a API do Twitter, utilizando os dados do arquivo json
api = twitter.Api(**data)

In [ ]:
# Testar se tudo está certo

# Caso tiver alguma credencial errada, o resultado será:
#twitter.error.TwitterError: [{'code': 32, 'message': 'Could not authenticate you.'}]

print(api.VerifyCredentials())

In [ ]:
# A API do Twitter permite fazer diversos tipos de consultas. O retorno dessas consultas pode ser acessado facilmente. 

# Função que, a partir de uma lista de status, mostra o nome do usuário e o texto do twitter.

def print_status(status_list):
    for status in status_list:
        print('(' + str(status.created_at) + ') ' + str(status.user.name) + ':' + str(status.text) + '\n')

## Buscando dados do twitter

In [ ]:
# Busca por termos

status_list = api.GetSearch(term="mi band 4",
                            lang='pt',
                            count=100)  #,
                            #result_type='mixed')

## Criando lista com os tweets

In [ ]:
def cria_tweet_list(status_list):
    tweet_list = []
    
    for tweet in status_list:
        tweet_list.append(tweet.text)
    
    return tweet_list

In [ ]:
# Printando elementos
print_status(status_list=status_list)

In [ ]:
print(len(status_list))

In [ ]:
tweet_list = cria_tweet_list(status_list)

In [ ]:
print(tweet_list)

In [ ]:
for i in tweet_list:
    print("'" + str(i) + "'")
    print('----------------------------------------------------------------------------------------------------------------')

In [ ]:
# criei
# tweet_list
# tweet_list_2
# tweet_list_3

## Remove os tweets duplicados

In [ ]:
lista_sem_duplicados = list(set(tweet_list))

In [ ]:
print(type(lista_sem_duplicados))

In [ ]:
print(len(tweet_list))

In [ ]:
print(len(lista_sem_duplicados))

## Coloca os tweets da lista em um arquivo .csv

In [ ]:
def cria_arquivo_csv(lista):
    arquivo = open('tweets_mi_band_4.csv', 'w', encoding='utf-8')
    
    for pos_tweet in range(len(lista)):
        arquivo.write('"' + str(lista[pos_tweet]) + '"')
        arquivo.write('\n')
        
    arquivo.close()

In [ ]:
cria_arquivo_csv(lista_sem_duplicados)

## Cria um data frame

In [51]:
# Criando data frame
df = pd.read_csv('classificado_tweets_mi_band_4.csv', encoding='utf-8')

In [ ]:
df = pd.read_csv('Tweets_Mg.csv', encoding='utf-8')

In [52]:
df.head()

,tweets,classes
0,Tive que desativar o negócio do sono da mi ban...,neutro
1,"RT @cachorro_frio: @fernandobentto cara, jura?...",negativo
2,"@FelipeDuarte151 Tenho a mi band 3, mas acho a...",positivo
3,To vendendo minha pulseira mi band 4(relógio)😉,neutro
4,mi band 4 na internet = 160 reais \nmi band 3 ...,neutro


In [53]:
tweets = df['tweets']
classes = df['classes']

In [36]:
tweets = df['Text']
classes = df['Classificacao']

## Limpeza dos dados

In [7]:
# Para limpeza dos dados
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

In [9]:
Preprocessing('Eu não gosto do partido, e também não votaria novamente nesse governante! http://python.w3.pt/?p=234')

'não gost partido, não vot nov ness governante!'

## Tokenização

In [10]:
from nltk.tokenize import TweetTokenizer

In [11]:
tweet_tokenizer = TweetTokenizer()

In [ ]:
frase = 'A live do @blogminerando é show! :) :-) ;) =D'

In [ ]:
tweet_tokenizer.tokenize(frase)

## Vetorização - Bag of words

In [54]:
# Instancia o objeto que faz a vectorização
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [55]:
# Aplica o vetorizador nos dados
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

scipy.sparse.csr.csr_matrix

In [56]:
# Formato linha, coluna da matriz
freq_tweets.shape

(93, 580)

In [57]:
# Treino do modelo de machine learning
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [58]:
freq_tweets.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Testando o modelo

In [59]:
# Testando o modelo
testes = ["Eu comprei um mi band 4",
         "Gostei do mi band 4",
         "Não quero o mi band 4",
         "Amei meu mi band 4",
         "Quero muito um mi band 4",
         "Alguem me de um mi band 4 por favor",
         "Oi, estou vendendo esse Xiaomi Mi Band 4 novinho",
         "acho q vou comprar o mi band 4",
         "O mi band 4 é muito ruim, não gostei",
         "Acho que esse  mi band quebra fácil, não gostei",
          ]

In [37]:
# Testando o modelo com dados do minerando dados
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é mais uma vez do PT']

In [18]:
# Aplicando a função de pre processamento nos testes
testes = [Preprocessing(i) for i in testes]

In [60]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [61]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

Eu comprei um mi band 4, positivo
Gostei do mi band 4, positivo
Não quero o mi band 4, positivo
Amei meu mi band 4, neutro
Quero muito um mi band 4, positivo
Alguem me de um mi band 4 por favor, positivo
Oi, estou vendendo esse Xiaomi Mi Band 4 novinho, positivo
acho q vou comprar o mi band 4, positivo
O mi band 4 é muito ruim, não gostei, positivo
Acho que esse  mi band quebra fácil, não gostei, positivo


In [62]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

['negativo' 'neutro' 'positivo']


array([[0.  , 0.15, 0.85],
       [0.  , 0.46, 0.54],
       [0.  , 0.19, 0.81],
       [0.  , 0.55, 0.45],
       [0.  , 0.07, 0.93],
       [0.  , 0.07, 0.93],
       [0.  , 0.46, 0.54],
       [0.  , 0.04, 0.96],
       [0.  , 0.06, 0.94],
       [0.  , 0.23, 0.77]])

## Criando modelos com Pipelines

In [22]:
# Pipelines são interessantes para reduzir código e automatizar fluxos
from sklearn.pipeline import Pipeline

In [63]:
pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

In [64]:
pipeline_simples.fit(tweets,classes)

Pipeline(memory=None,
         steps=[('counts',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [65]:
pipeline_simples.steps

[('counts',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=1.0, max_features=None, min_df=1,
                  ngram_range=(1, 1), preprocessor=None, stop_words=None,
                  strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [66]:
pipeline_svm_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', svm.SVC(kernel='linear'))
])

## Validando os Modelos com Validação Cruzada

In [67]:
# Fazendo o cross validation do modelo
resultados = cross_val_predict(pipeline_simples, tweets, classes, cv=10)

c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [68]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(classes,resultados)

0.6344086021505376

In [69]:
# Medidas de validação do modelo
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

              precision    recall  f1-score   support

    Positivo       0.00      0.00      0.00         0
    Negativo       0.00      0.00      0.00         0
      Neutro       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00         0
   macro avg       0.00      0.00      0.00         0
weighted avg       0.00      0.00      0.00         0



c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and 

In [70]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   negativo  neutro  positivo  All
Real                                     
negativo         1       2         3    6
neutro           0      29        19   48
positivo         0      10        29   39
All              1      41        51   93


In [71]:
def Metricas(modelo, tweets, classes):
  resultados = cross_val_predict(modelo, tweets, classes, cv=10)
  return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

In [72]:
# svm linear simples
Metricas(pipeline_svm_simples,tweets,classes)

c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


'Acurácia do modelo: 0.6774193548387096'

In [73]:
# svm linear simples
Metricas(pipeline_simples,tweets,classes)

c:\users\maluf\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


'Acurácia do modelo: 0.6344086021505376'